## Data Science from Scratch

#### Meeting: Aug 17, 2019 @ Cinco Ranch Library, Classroom on 2nd Floor

Reading through and working exercises from the Second Edition of Joel Grus' classic book, Data Science from Scratch: First Priciples from Python

## Chapter 13 - Naive Bayes

### Naive Bayes Classifier

In [3]:
from typing import Set
import re

def tokenize(text: str) -> Set[str]:
    text = text.lower()
    all_words = re.findall("[a-z0-9']+", text)
    return set(all_words)

assert tokenize("Data Science is science") == {"data", "science", "is"}
    

In [4]:
from typing import NamedTuple

class Message(NamedTuple):
    text: str
    is_spam: bool
       

In [5]:
from typing import List, Tuple, Dict, Iterable
import math
from collections import defaultdict

class NaiveBayesClassifier:
    def __init__(self, k: float = 0.5) -> None:
        self.k = k
        
        self.tokens: Set[str] = set()
        self.token_spam_counts: Dict[str, int] = defaultdict(int)
        self.token_ham_counts: Dict[str, int] = defaultdict(int)
        self.spam_messages = self.ham_messages = 0
        
    def train(self, messages: Iterable[Message]) -> None:
        for message in messages:
            # inc message counts
            if message.is_spam:
                self.spam_messages += 1
            else:
                self.ham_messages += 1
                
            # inc word counts
            for token in tokenize(message.text):
                self.tokens.add(token)
                if message.is_spam:
                    self.token_spam_counts[token] += 1
                else:
                    self.token_ham_counts[token] += 1
                    
    def _probabilities(self, token: str) -> Tuple[float, float]:
        """returns P(token | spam), P(token | ham)"""
        spam = self.token_spam_counts[token]
        ham = self.token_ham_counts[token]
        
        p_token_spam = (spam + self.k) / (self.spam_messages + 2 * self.k)
        p_token_ham = (ham + self.k) / (self.ham_messages + 2 * self.k)
        
        return p_token_spam, p_token_ham
        
    def predict(self, text: str) -> float:
        text_tokens = tokenize(text)
        log_prob_if_spam = log_prob_if_ham = 0.0
        
        # iterate through each word in out vocab
        for token in self.tokens:
            prob_if_spam, prob_if_ham = self._probabilities(token)
            
            # if token appears in message, add log prob
            if token in text_tokens:
                log_prob_if_spam += math.log(prob_if_spam)
                log_prob_if_ham += math.log(prob_if_ham)
                
            # otherwise add log prob _not_ seeing it, log(1 - prob of seeing it)
            else:
                log_prob_if_spam += math.log(1.0 -  prob_if_spam)
                log_prob_if_ham += math.log(1.0 -  prob_if_ham)
                
        prob_if_spam = math.exp(log_prob_if_spam)
        prob_if_ham = math.exp(log_prob_if_ham)
        return prob_if_spam / (prob_if_spam + prob_if_ham)
                

### Testing

In [6]:
messages = [Message("spam rules", is_spam=True),
            Message("ham rules", is_spam=False),
            Message("hello ham", is_spam=False)]

model = NaiveBayesClassifier(k=0.5)
model.train(messages)

assert model.tokens == {"spam", "ham", "rules", "hello"}
assert model.spam_messages == 1
assert model.ham_messages == 2
assert model.token_spam_counts == {"spam": 1, "rules": 1}
assert model.token_ham_counts == {"ham": 2, "rules": 1, "hello": 1}


text = "hello spam"
assert 0.83 < model.predict(text) < 0.84
print (model.predict(text)) 

0.8350515463917525


### Using the model

In [2]:
#data

from io import BytesIO
import requests
import tarfile
BASE_URL = "https://spamassassin.apache.org/old/publiccorpus/"

FILES = ["20021010_easy_ham.tar.bz2",
         "20021010_hard_ham.tar.bz2",
         "20021010_spam.tar.bz2"]

OUTPUTDIR = "c:\\github\\spam_data\\"

for filename in FILES:
        content = requests.get(f"{BASE_URL}/{filename}").content
        fin = BytesIO(content)
        
        with tarfile.open(fileobj=fin, mode='r:bz2') as tf:
            tf.extractall(OUTPUTDIR)

In [8]:
import glob, re

path = "c:\\github\\spam_data\\*\\*"

data: List[Message] = []
for filename in glob.glob(path):
    is_spam = "ham" not in filename
    
    with open (filename, errors='ignore') as email_file:
        for line in email_file:
            if line.startswith("Subject:"):
                subject = line.lstrip("Subject: ")
                data.append(Message(subject, is_spam))
                break

In [11]:
import random
from scratch.machine_learning import split_data

random.seed(0)
train_messages, test_messages = split_data(data, 0.75)

model = NaiveBayesClassifier ()
model.train(messages=train_messages)

In [14]:
from collections import Counter
predictions = [(message, model.predict(message.text)) for message in test_messages]

confusion_matrix = Counter((message.is_spam, spam_probability > 0.5) for message, spam_probability in predictions)

print(confusion_matrix)

Counter({(False, False): 670, (True, True): 86, (True, False): 40, (False, True): 29})


In [15]:
def p_spam_given_token(token: str, model: NaiveBayesClassifier) -> float:
    prob_if_spam, prob_if_ham = model._probabilities(token)
    return prob_if_spam / (prob_if_spam + prob_if_ham)

words = sorted(model.tokens, key=lambda t: p_spam_given_token(t, model))

print("spammiest_words", words[-10:])
print("hammiest_words", words[:10])


spammiest_words ['assistance', '95', 'attn', 'clearance', 'money', 'per', 'sale', 'rates', 'systemworks', 'adv']
hammiest_words ['spambayes', 'users', 'razor', 'zzzzteana', 'sadev', 'apt', 'perl', 'ouch', 'spamassassin', 'bliss']
